In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from PIL import Image

In [168]:
df = pd.read_csv('all_cells_tpm.csv')
print(df.columns)
mode = 'summary'
search_term = 'heat'
groups = ['slow', 'fast', '22c06_', 'PF', 'AC']

Index(['FB', 'slow1', 'slow2', 'slow3', 'fast1', 'fast2', 'fast3', '22c06_1',
       '22c06_2', '22c06_3', '22c06_4', 'PF1', 'PF2', 'PF3', 'PF4', 'AC1',
       'AC2', 'AC3', 'name', 'symbol', 'go', 'summary'],
      dtype='object')


In [169]:
extract_groups = [[t for t in df.columns if g in t] for g in groups]
fb_list = []
for ind in df.index:
    try:
        if search_term in df[mode][ind]:
            fb_list.append(df['FB'][ind])
    except:
        pass

for k,fb in enumerate(fb_list):
    test = [np.array(df[df.FB == fb][g]).flatten() for g in extract_groups]
    g = sns.boxplot(test)
    g.set_xticklabels(groups)
    g.set_title(df[df.FB == fb].name.item())
    plt.savefig('temp/Test'+str(k)+'.jpg')
    plt.close()

# stitch images together
image_list = ['temp/Test'+str(k)+'.jpg' for k in range(len(fb_list))]
max_stitch = 10
tot_images = len(image_list)

images = [Image.open(x) for x in image_list]
width, height = images[0].size

total_width = max_stitch*width
max_height = (tot_images//max_stitch + 1*(tot_images%max_stitch>0))*height

new_im = Image.new('RGB', (total_width, max_height))

for k,im in enumerate(images):
    x,y = (k//max_stitch),(k%max_stitch)
    new_im.paste(im, (y*width,x*height))

new_im.save('image1.jpg')


# DEPRECATED

In [3]:
df = pd.read_csv('gene_association.fb', sep='\t',skiprows=5, header=None)
df = df[[1,4]]
df.columns=['fb','go']

list_of_fbs = set(df.fb)
go_dict = {fb: [] for fb in list_of_fbs}


for ind in df.index:
    go_dict[df['fb'][ind]].append(df['go'][ind])

for fb, go in go_dict.items():
    go_dict[fb] = ';'.join(go)

In [4]:
df = pd.read_csv('best_gene_summary_fb_2023_04.tsv', sep='\t',skiprows=8)
list_of_fbs = set(df['#FBgn_ID'])
sum_dict = {fb: 'o' for fb in list_of_fbs}

for ind in df.index:
    sum_dict[df['#FBgn_ID'][ind]] = (df['Summary'][ind])

In [ ]:
df = pd.read_csv('all_cells_tpm.csv')
df=df.rename(columns = {'Unnamed: 0':'FB'})
df["go"] = np.nan
df["summary"] = np.nan

In [45]:
for ind in df.index:
    try:
        df.loc[ind,'go'] = go_dict[df['FB'][ind]]
    except:
        df.loc[ind,'go'] = np.nan
        
    try:
        df.loc[ind,'summary'] = sum_dict[df['FB'][ind]]
    except:
        df.loc[ind,'summary'] = np.nan

In [126]:
df.to_csv('all_cells_tpm.csv', index=False)